In [ ]:
#@title Funções e bibliotecas

!apt-get install fluidsynth &> /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2 &> /dev/null
!apt-get install lilypond > /dev/null
!sudo apt-get install musescore >/dev/null

from IPython.display import Audio
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from music21 import *

def play(music):
  filename = music.write('mid')
  !fluidsynth -ni font.sf2 $filename -F $filename\.wav -r 16000 > /dev/null
  display(Audio(filename + '.wav'))

def show(music):
  return Image(filename=str(music.write('lily.png')))

us = environment.UserSettings()
us['lilypondPath'] = '/usr/bin/lilypond'
us['musescoreDirectPNGPath'] = '/usr/bin/musescore'
us['musicxmlPath'] = '/usr/bin/musescore'

def create_scale(scale_pitch, mode):
  s = stream.Stream()
  if mode == 'major':
    ps = scale.MajorScale(scale_pitch).pitches
  else:
    ps = scale.MinorScale(scale_pitch).pitches
  for p in range(len(ps)*2):
    if p == 8: continue
    n = note.Note()
    n.pitch = pitch.Pitch(ps[p%len(ps)].name)
    n.quarterLength = 1.0
    if p < 8:
      n.octave = 4
    else:
      n.octave = 5
    if n.pitch.name == scale_pitch and p > 0: n.octave += 1
    s.append(n)
  return s

import random

def get_chord(scale_pitch, mode):
  notes = [n for n in create_scale(scale_pitch, mode).notes]
  notes = notes[:len(notes)//2]
  f = random.randint(0, len(notes)-1)
  c = chord.Chord([notes[f], notes[(f+2)%len(notes)], notes[(f+4)%len(notes)]])
  return c

def random_scale_chord():
  ps = ['A', 'B', 'C', 'E', 'F', 'G']
  sh = [p+'#' for p in ps]
  b = [p+'b' for p in ps]
  ps = ps + b + sh

  rand_p = ps[random.randint(0,len(ps)-1)]
  m = random.randint(0,1)
  if m == 0:
    mode = 'major'
    ps = scale.MajorScale(rand_p).pitches
  else:
    mode = 'minor'
    ps = scale.MinorScale(rand_p).pitches
  
  s = stream.Stream()
  for p in ps:
    n = note.Note()
    n.pitch = p
    n.quarterLength = 1.0
    s.append(n)

  c = stream.Stream()
  c.append(key.Key(rand_p, mode))
  c.append(get_chord(rand_p, mode))

  return s, c

def random_chord():
  ps = ['A', 'B', 'C', 'E', 'F', 'G']
  sh = [p+'#' for p in ps]
  b = [p+'b' for p in ps]
  ps = ps + b + sh

  m = random.randint(0,1)
  if m == 1:
    mode = 'major'
  else:
    mode = 'minor'

  rand_p = ps[random.randint(0,len(ps)-1)]
  c = get_chord(rand_p, mode)
  
  return c

record_ex1 = 0
record_ex2 = 0

In [ ]:
#@title Análise da estrutura harmônica das tríades
l = [i for i in corpus.chorales.ChoraleList().byBudapest]

triads = 0
root_triads = 0
inv_triads = 0
first_inv = 0
second_inv = 0

types = []
degrees = []
doubles = []

for indice in l:
  c = corpus.parse('bach/bwv' + str(corpus.chorales.ChoraleList().byBudapest[indice]['bwv']))

  for i in c.chordify().flat.notesAndRests:
    if i.isChord:
      s = stream.Stream()
      s.append(c.analyze('key'))
      s.append(i)
      n = [p.name for p in i.pitches]
      if len(set(n)) == 3: 
        degrees.append(i.scaleDegrees[i.pitches.index(i.root())][0])
        types.append(i.commonName)
        triads += 1
        if i.bass() != i.root():
          inv_triads += 1
          if i.bass() == i.third:
            first_inv += 1
          elif i.bass() == i.fifth:
            second_inv += 1
        else: root_triads += 1
        try:
          d = [n.count(i.root().name), n.count(i.third.name), n.count(i.fifth.name)]
          m = d.index(max(d))
          if m == 0: doubles.append('fundamental')
          elif m == 1: doubles.append('terça')
          elif m == 2: doubles.append('quinta')
        except: pass


print('Número de tríades:', triads)
print('|__ '+str(round(100*root_triads/triads, 2)) + '% --- fundamental')
print('|__ '+str(round(100*inv_triads/triads, 2)) +  '% --- invertidas')
print('    |__ '+str(round(100*first_inv/inv_triads, 2)) +  '% ---- primeira inversão')
print('    |__ '+str(round(100*second_inv/inv_triads, 2)) +  '% --- segunda inversão')

from collections import Counter
type_counts = Counter(types).most_common(10)
degree_counts = Counter(degrees).most_common(100)
doubles_count = Counter(doubles).most_common(100)

import plotly.express as px
bp = px.bar(y=[i for i,j in type_counts], x=[j for i,j in type_counts], height=300, width=700, orientation='h', title='Top 10 tipos de tríade mais comuns')
bp.show()
bp2 = px.bar(x=[i for i,j in degree_counts], y=[j for i,j in degree_counts], height=300, width=700, title='Distribuição dos graus da escala')
bp2.show()
bp3 = px.bar(x=[i for i,j in doubles_count], y=[j for i,j in doubles_count], height=300, width=700, title='Dobramentos mais comuns')
bp3.show()

Número de tríades: 10746
|__ 59.6% --- fundamental
|__ 40.4% --- invertidas
    |__ 70.95% ---- primeira inversão
    |__ 23.82% --- segunda inversão


Ideias:
- qtd consonantes vs dissonantes